In [1]:
import sys
from datasets import load_dataset
import torch
from trl.trainer.dpo_trainer import DPOTrainer
import gc

sys.path.append('..')
from model.causal_lm import AutoLMFineTuner
from model.utils import LMHyperParams, SmModel, ModelChoice
from train import start_training
from dataset.squad import UltraFeedbackDataModule
%load_ext autoreload
%autoreload 2

/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-15 01:45:36.101 | INFO     | train:<module>:5 - Loading dependencies - Torch...
2024-11-15 01:45:36.125 | INFO     | train:<module>:16 - Loading dependencies - Lightning...
2024-11-15 01:45:36.125 | INFO     | train:<module>:25 - Loading dependencies - Project...


In [2]:
params = LMHyperParams(
    base_model_checkpoint="Qwen/Qwen2.5-0.5B",
    warmup_steps_count=10,
    tuning_type="dpo",
    train_batch_size=1,
    gradient_accumulation_steps=1,
    optimizer="AdamW",
)

# https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/dpo-align-llms-in-2024-with-trl.ipynb

In [3]:
gc.collect()
torch.cuda.empty_cache()

In [4]:
model = AutoLMFineTuner(params)
model.to('cuda')

2024-11-15 01:45:42.941 | INFO     | model.causal_lm:__init__:69 - Using DPO with LoraConfig: LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=256, target_modules='all-linear', lora_alpha=128, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))


AutoLMFineTuner(
  (model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): Qwen2ForCausalLM(
        (model): Qwen2Model(
          (embed_tokens): Embedding(151936, 896)
          (layers): ModuleList(
            (0-23): 24 x Qwen2DecoderLayer(
              (self_attn): Qwen2SdpaAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=896, out_features=896, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=896, out_features=256, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=256, out_features=896, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                  (lora_magnitude_vec

In [5]:
data_module = UltraFeedbackDataModule(2, model.tokenizer, 1024)
data_module.cpu_count = 12
data_module.setup("fit")

2024-11-15 01:45:50.087 | INFO     | dataset.squad:setup:214 - Loading dataset for stage fit


2024-11-15 01:45:52.074 | INFO     | dataset.squad:setup:222 - Processing dataset for stage fit, workers: 12, cache dir dataset_caches/ultrafeedback


In [7]:
for batch in data_module.train_dataloader():
    for k, v in batch.items():
        batch[k] = v.to('cuda')
    loss = model._step(batch)
    print(loss)
    break

tensor(0., device='cuda:0', grad_fn=<MeanBackward0>)


In [ ]:
start_training(
    "16-mixed", "", model, model.tokenizer, data_module, params, False, ""
)
